# Imports

In [74]:
import os
from tqdm.notebook import tqdm
import xml.etree.ElementTree as ET
import subprocess

# XML Parser

In [75]:
def replace_key_in_xml(xml_file, key, replacement_value, output_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for elem in root.iter():
        if (elem.tag == key and elem.text == "Power4") or elem.tag == "BenchmarkPath":
            elem.text = replacement_value
    tree.write(output_file)

# Build and Run

In [76]:
def run(file_name):
    stat_file = f"./outputs/{file_name}.stat"
    err_file = f"./err/{file_name}.err"
    assignment_dir="/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5"
    resource_dir = "/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources"
    cmd = f"java -jar {resource_dir}/Tejas/jars/tejas.jar {assignment_dir}/config_tigerlake.xml {assignment_dir}/outputs/{stat_file}.stat"
    with open(err_file, "w") as output_file: 
        subprocess.run(cmd, shell=True, stdout=output_file)

# Constants

In [77]:
prefetchers = ["None", "Simple", "Power4", "Bingo"]
config_file_path = "./config_tigerlake.xml"
output_file = "./config_tigerlake_modified.xml"

# New Benchmarks

In [78]:
benchmark_path = "/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/phoronix_short_traces/"
benchmarks = os.listdir(benchmark_path)

total = len(prefetchers) * len(benchmarks)
pb = tqdm(total=total)
for prefetcher in prefetchers:
    replace_key_in_xml(config_file_path, "Prefetcher", prefetcher, output_file)
    for benchmark in benchmarks:
        pb.update(1)
        if "_8" not in benchmark:
            continue
        print(benchmark)
        replace_key_in_xml(output_file, "BenchmarkPath",
                           benchmark_path+benchmark+f"/{benchmark.split('_')[0]}", output_file)
        file_name = f"{benchmark}_{prefetcher}"
        run(file_name)
    

  0%|          | 0/32 [00:00<?, ?it/s]

stream_8


 (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits) 

KeyboardInterrupt: 